In [1]:
import tensorflow as tf
from tensorflow import keras
from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time
from sklearn.model_selection import train_test_split
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from sklearn.metrics import classification_report
from sklearn.feature_extraction import image
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
nc_frmse = 'APP-x.1400.ERA-I.1982-2018.07.Arctic.RMSE.5V.RAND.100epoch_leaky.nc'
h5_model = 'APP-x.1400.ERA-I.1982-2018.07.Arctic.1982-2009.5V.RAND.100epoch_leaky.h5'
nc_fy    = 'APP-x.1400.ERA-I.1982-2018.07.Arctic.2010-2018.nc'
nc_fay   = 'APP-x.1400.ERA-I.1982-2018.07.anom.wrt.2001-2018.Arctic.2010-2018.nc'
nc_fm    = 'APP-x.1400.ERA-I.1982-2018.06.Arctic.2010-2018.nc'
nc_fam   = 'APP-x.1400.ERA-I.1982-2018.06.anom.wrt.2001-2018.Arctic.2010-2018.nc'

In [3]:
psize  = 11
cx     = int((psize-1)/2)
cy     = int((psize-1)/2)

In [4]:
# input file for SIC Yr-1

nc_fidy  = Dataset(nc_fy, 'r')

time    = nc_fidy.variables['time'][:]

ysic0   = nc_fidy.variables['icecon'][:]
ythk0   = nc_fidy.variables['t2m'][:]
yvtn0   = nc_fidy.variables['v10'][:]
ytem0   = nc_fidy.variables['sst'][:]
yalb0   = nc_fidy.variables['fal'][:]

# input file for SIC, THK, Temperature and ALBEDO Mon-1

nc_fidm = Dataset(nc_fm, 'r')
msic0   = nc_fidm.variables['icecon'][:]
mthk0   = nc_fidm.variables['t2m'][:]
mvtn0   = nc_fidm.variables['v10'][:]
mtem0   = nc_fidm.variables['sst'][:]
malb0   = nc_fidm.variables['fal'][:]

# input file for SIC Anomaly, Yr-1

nc_fiday = Dataset(nc_fay, 'r')
aysic0   = nc_fiday.variables['icecon'][:]
aythk0   = nc_fiday.variables['t2m'][:]
ayvtn0   = nc_fiday.variables['v10'][:]
aytem0   = nc_fiday.variables['sst'][:]
ayalb0   = nc_fiday.variables['fal'][:]

# input file for SIC Anomaly, Mon-1

nc_fidam = Dataset(nc_fam, 'r')
amsic0   = nc_fidam.variables['icecon'][:]
amthk0   = nc_fidam.variables['t2m'][:]
amvtn0   = nc_fidam.variables['v10'][:]
amtem0   = nc_fidam.variables['sst'][:]
amalb0   = nc_fidam.variables['fal'][:]

In [5]:
ysic1   = np.transpose(ysic0)
ythk1   = np.transpose(ythk0)
yvtn1   = np.transpose(yvtn0)
ytem1   = np.transpose(ytem0)
yalb1   = np.transpose(yalb0)

msic1   = np.transpose(msic0)
mthk1   = np.transpose(mthk0)
mvtn1   = np.transpose(mvtn0)
mtem1   = np.transpose(mtem0)
malb1   = np.transpose(malb0)

aysic1   = np.transpose(aysic0)
aythk1   = np.transpose(aythk0)
ayvtn1   = np.transpose(ayvtn0)
aytem1   = np.transpose(aytem0)
ayalb1   = np.transpose(ayalb0)

amsic1   = np.transpose(amsic0)
amthk1   = np.transpose(amthk0)
amvtn1   = np.transpose(amvtn0)
amtem1   = np.transpose(amtem0)
amalb1   = np.transpose(amalb0)


print("msic1.shape:", msic1.shape)
print("ysic1.shape:", ysic1.shape)

msic1.shape: (361, 361, 9)
ysic1.shape: (361, 361, 9)


In [6]:
print, time

(<function print>,
 masked_array(data=[1982.    , 1982.0834, 1982.1666, 1982.25  , 1982.3334,
                    1982.4166, 1982.5   , 1982.5834, 1982.6666, 1982.75  ,
                    1982.8334, 1982.9166, 1983.    , 1983.0834, 1983.1666,
                    1983.25  , 1983.3334, 1983.4166, 1983.5   , 1983.5834,
                    1983.6666, 1983.75  , 1983.8334, 1983.9166, 1984.    ,
                    1984.0834, 1984.1666, 1984.25  , 1984.3334, 1984.4166,
                    1984.5   , 1984.5834, 1984.6666, 1984.75  , 1984.8334,
                    1984.9166, 1985.    , 1985.0834, 1985.1666, 1985.25  ,
                    1985.3334, 1985.4166, 1985.5   , 1985.5834, 1985.6666,
                    1985.75  , 1985.8334, 1985.9166, 1986.    , 1986.0834,
                    1986.1666, 1986.25  , 1986.3334, 1986.4166, 1986.5   ,
                    1986.5834, 1986.6666, 1986.75  , 1986.8334, 1986.9166,
                    1987.    , 1987.0834, 1987.1666, 1987.25  , 1987.3334,
      

In [7]:
# Remove NaNs
Y_SIC2   = np.nan_to_num(ysic1[:,:,1:])
Y_time   = time[1:]

ysic2    = np.nan_to_num(ysic1[:,:,:-1])
ythk2    = np.nan_to_num(ythk1[:,:,:-1])
yvtn2    = np.nan_to_num(yvtn1[:,:,:-1])
ytem2    = np.nan_to_num(ytem1[:,:,:-1])
yalb2    = np.nan_to_num(yalb1[:,:,:-1])


msic2    = np.nan_to_num(msic1[:,:,1:])
mthk2    = np.nan_to_num(mthk1[:,:,1:])
mvtn2    = np.nan_to_num(mvtn1[:,:,1:])
mtem2    = np.nan_to_num(mtem1[:,:,1:])
malb2    = np.nan_to_num(malb1[:,:,1:])

aysic2    = np.nan_to_num(aysic1[:,:,:-1])
aythk2    = np.nan_to_num(aythk1[:,:,:-1])
ayvtn2    = np.nan_to_num(ayvtn1[:,:,:-1])
aytem2    = np.nan_to_num(aytem1[:,:,:-1])
ayalb2    = np.nan_to_num(ayalb1[:,:,:-1])

amsic2    = np.nan_to_num(amsic1[:,:,1:])
amthk2    = np.nan_to_num(amthk1[:,:,1:])
amvtn2    = np.nan_to_num(amvtn1[:,:,1:])
amtem2    = np.nan_to_num(amtem1[:,:,1:])
amalb2    = np.nan_to_num(amalb1[:,:,1:])

In [8]:
Y_SIC3 = image.extract_patches_2d(Y_SIC2, (psize, psize))

ysic3  = image.extract_patches_2d(ysic2,  (psize, psize))
ythk3  = image.extract_patches_2d(ythk2,  (psize, psize))
yvtn3  = image.extract_patches_2d(yvtn2,  (psize, psize))
ytem3  = image.extract_patches_2d(ytem2,  (psize, psize))
yalb3  = image.extract_patches_2d(yalb2,  (psize, psize))

msic3  = image.extract_patches_2d(msic2,  (psize, psize))
mthk3  = image.extract_patches_2d(mthk2,  (psize, psize))
mvtn3  = image.extract_patches_2d(mvtn2,  (psize, psize))
mtem3  = image.extract_patches_2d(mtem2,  (psize, psize))
malb3  = image.extract_patches_2d(malb2,  (psize, psize))

amsic3 = image.extract_patches_2d(amsic2, (psize, psize))
amthk3 = image.extract_patches_2d(amthk2, (psize, psize))
amvtn3 = image.extract_patches_2d(amvtn2, (psize, psize))
amtem3 = image.extract_patches_2d(amtem2, (psize, psize))
amalb3 = image.extract_patches_2d(amalb2, (psize, psize))

aysic3 = image.extract_patches_2d(aysic2, (psize, psize))
aythk3 = image.extract_patches_2d(aythk2, (psize, psize))
ayvtn3 = image.extract_patches_2d(ayvtn2, (psize, psize))
aytem3 = image.extract_patches_2d(aytem2, (psize, psize))
ayalb3 = image.extract_patches_2d(ayalb2, (psize, psize))

In [9]:
print(" ysic3 shape:", ysic3.shape)
print(" msic3 shape:", msic3.shape)
print("aysic3 shape:", aysic3.shape)
print("amsic3 shape:", amsic3.shape)

 ysic3 shape: (123201, 11, 11, 8)
 msic3 shape: (123201, 11, 11, 8)
aysic3 shape: (123201, 11, 11, 8)
amsic3 shape: (123201, 11, 11, 8)


In [10]:
Y_SIC  = np.transpose(Y_SIC3,(1,2,3,0))

ysic   = np.transpose(ysic3,(1,2,3,0))
ythk   = np.transpose(ythk3,(1,2,3,0))
yvtn   = np.transpose(yvtn3,(1,2,3,0))
ytem   = np.transpose(ytem3,(1,2,3,0))
yalb   = np.transpose(yalb3,(1,2,3,0))

msic   = np.transpose(msic3,(1,2,3,0))
mthk   = np.transpose(mthk3,(1,2,3,0))
mvtn   = np.transpose(mvtn3,(1,2,3,0))
mtem   = np.transpose(mtem3,(1,2,3,0))
malb   = np.transpose(malb3,(1,2,3,0))

aysic   = np.transpose(aysic3,(1,2,3,0))
aythk   = np.transpose(aythk3,(1,2,3,0))
ayvtn   = np.transpose(ayvtn3,(1,2,3,0))
aytem   = np.transpose(aytem3,(1,2,3,0))
ayalb   = np.transpose(ayalb3,(1,2,3,0))

amsic   = np.transpose(amsic3,(1,2,3,0))
amthk   = np.transpose(amthk3,(1,2,3,0))
amvtn   = np.transpose(amvtn3,(1,2,3,0))
amtem   = np.transpose(amtem3,(1,2,3,0))
amalb   = np.transpose(amalb3,(1,2,3,0))


print("Y_SIC.shape:", Y_SIC.shape)
print("ysic.shape:", ysic.shape)
print("msic.shape:", msic.shape)

Y_SIC.shape: (11, 11, 8, 123201)
ysic.shape: (11, 11, 8, 123201)
msic.shape: (11, 11, 8, 123201)


In [11]:
px, py, nyr, npatch = ysic.shape

aysic_2d = aysic.reshape((px*py, nyr*npatch))
amsic_2d = amsic.reshape((px*py, nyr*npatch))
mthk_2d  =  mthk.reshape((px*py, nyr*npatch))
mvtn_2d  =  mvtn.reshape((px*py, nyr*npatch))
malb_2d  =  malb.reshape((px*py, nyr*npatch))
mtem_2d  =  mtem.reshape((px*py, nyr*npatch))


print("aysic_2d.shape:", aysic_2d.shape)
print("amsic_2d.shape:", amsic_2d.shape)
print("mthk_2d.shape :", mthk_2d.shape)
print("mvtn_2d.shape :", mvtn_2d.shape)
print("malb_2d.shape :", malb_2d.shape)
print("mtem_2d.shape :", mtem_2d.shape)

aysic_2d.shape: (121, 985608)
amsic_2d.shape: (121, 985608)
mthk_2d.shape : (121, 985608)
mvtn_2d.shape : (121, 985608)
malb_2d.shape : (121, 985608)
mtem_2d.shape : (121, 985608)


In [12]:
#normalization
scaler = StandardScaler()

naysic_2d = scaler.fit_transform(aysic_2d)
namsic_2d = scaler.fit_transform(amsic_2d)
nmthk_2d  = scaler.fit_transform(mthk_2d)
nmvtn_2d  = scaler.fit_transform(mvtn_2d)
nmalb_2d  = scaler.fit_transform(malb_2d)
nmtem_2d  = scaler.fit_transform(mtem_2d)
nysic  =  ysic.reshape((px, py, nyr*npatch))/100
nmsic  =  msic.reshape((px, py, nyr*npatch))/100

print("naysic_2d.shape:", naysic_2d.shape)
print("namsic_2d.shape:", namsic_2d.shape)
print("nmthk_2d.shape :", nmthk_2d.shape)
print("nmvtn_2d.shape :", nmvtn_2d.shape)
print("nmalb_2d.shape :", nmalb_2d.shape)
print("nmtem_2d.shape :", nmtem_2d.shape)

naysic_2d.shape: (121, 985608)
namsic_2d.shape: (121, 985608)
nmthk_2d.shape : (121, 985608)
nmvtn_2d.shape : (121, 985608)
nmalb_2d.shape : (121, 985608)
nmtem_2d.shape : (121, 985608)


In [13]:
Y_SIC_Center = Y_SIC[cx,cy,:,:].reshape((nyr*npatch))

naysic = naysic_2d.reshape((px, py, nyr*npatch))
namsic = namsic_2d.reshape((px, py, nyr*npatch))
nmthk  =  nmthk_2d.reshape((px, py, nyr*npatch))
nmvtn  =  nmvtn_2d.reshape((px, py, nyr*npatch))
nmalb  =  nmalb_2d.reshape((px, py, nyr*npatch))
nmtem  =  nmtem_2d.reshape((px, py, nyr*npatch))

print("Y_SIC_Center.shape:", Y_SIC_Center.shape)

print("nysic.shape: ",  nysic.shape)
print("nmsic.shape: ",  nmsic.shape)
print("naysic.shape:", naysic.shape)
print("namsic.shape:", namsic.shape)
print("nmthk.shape: ",  nmthk.shape)
print("nmvtn.shape: ",  nmvtn.shape)
print("nmalb.shape: ",  nmalb.shape)
print("nmtem.shape: ",  nmtem.shape)

print("nysic.range:  ",nysic.min(), nysic.max())
print("nmsic.range:  ",nmsic.min(), nmsic.max())
print("naysic.range: ",naysic.min(), naysic.max())
print("namsic.range: ",namsic.min(), namsic.max())
print("nmthk.range:  ",nmthk.min(), nmthk.max())
print("nmvtn.range:  ",nmvtn.min(), nmvtn.max())
print("nmalb.range:  ",nmalb.min(), nmalb.max())
print("nmtem.range:  ",nmtem.min(), nmtem.max())

Y_SIC_Center.shape: (985608,)
nysic.shape:  (11, 11, 985608)
nmsic.shape:  (11, 11, 985608)
naysic.shape: (11, 11, 985608)
namsic.shape: (11, 11, 985608)
nmthk.shape:  (11, 11, 985608)
nmvtn.shape:  (11, 11, 985608)
nmalb.shape:  (11, 11, 985608)
nmtem.shape:  (11, 11, 985608)
nysic.range:   0.0 1.0
nmsic.range:   0.0 1.0
naysic.range:  -10.954452 10.954452
namsic.range:  -10.954452 10.954452
nmthk.range:   -7.4388204 7.285815
nmvtn.range:   -6.238906 6.625461
nmalb.range:   -10.954452 10.954452
nmtem.range:   -10.954446 10.954452


In [14]:
# Combine all the standardized data and reshape
X1 = np.stack((nysic,nmsic,naysic,namsic,nmthk,nmvtn,nmalb,nmtem))
print("X1.shape:", X1.shape)
X  = np.transpose(X1)
print("X.shape:", X.shape)

X1.shape: (8, 11, 11, 985608)
X.shape: (985608, 11, 11, 8)


In [15]:
model = tf.keras.models.load_model(h5_model)

In [16]:
predicted_SIC = model.predict(X)
predicted_SIC.shape

(985608, 1)

In [17]:
import os
os.system('cp '+ nc_fy + ' '+ nc_frmse )  
nc_fout    = nc_frmse

nc_fout_id = Dataset(nc_fout, 'r+')

icetgt    = np.empty(shape=ysic0.shape,dtype='float') 
icepdt    = np.empty(shape=ysic0.shape,dtype='float')
icerms    = np.empty(shape=ysic0[0,:,:].shape,dtype='float') 

In [18]:
ll = 0
for kz in range(1,          icetgt.shape[0]):
    for ix in range(cx,     icetgt.shape[2]-cx):
        for jy in range(cy, icetgt.shape[1]-cy):
            icetgt[kz,jy,ix] = Y_SIC_Center[ll]
            icepdt[kz,jy,ix] = predicted_SIC[ll,0]
            ll = ll + 1

In [19]:
icetgt1 = icetgt[1:,:,:]
icepdt1 = icepdt[1:,:,:]
iceerr  = icepdt1 - icetgt1

ntime, nlat, nlon  = icetgt1.shape

icetgt1_2d = icetgt1.reshape((ntime, nlat*nlon))
icepdt1_2d = icepdt1.reshape((ntime, nlat*nlon))

icerms_2d  = mean_squared_error(icetgt1_2d, icepdt1_2d, multioutput='raw_values', squared=False)
icerms     = icerms_2d.reshape((nlat,nlon))
icerms[icerms == 0] = 'nan'
icerms_mean = np.nanmean(icerms)

In [20]:
icetgt[np.isnan(ysic0)] = 'nan'
icepdt[np.isnan(ysic0)] = 'nan'
iceerr[np.isnan(ysic0[1:,:,:])] = 'nan'

In [21]:
print(Y_SIC_Center[0:icetgt.shape[2]*2-cx*4])
print(icetgt[1,     cy:icetgt.shape[1]-cy,        cx:cx+2])
print(icepdt[1,     cy:icetgt.shape[1]-cy,        cx:cx+2])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [22]:
NC_SIC_TGT         = nc_fout_id.createVariable('icetgt','f4',('time', 'num_pixels', 'num_lines'))
NC_SIC_PDT         = nc_fout_id.createVariable('icepdt','f4',('time', 'num_pixels', 'num_lines'))
NC_SIC_ERR         = nc_fout_id.createVariable('iceerr','f4',('time', 'num_pixels', 'num_lines'))
NC_SIC_RMS         = nc_fout_id.createVariable('icerms','f4',('time', 'num_pixels', 'num_lines'))
NC_SIC_RMS_MEAN    = nc_fout_id.createVariable('icerms_mean','f4',('time'))

NC_SIC_TGT[:,:,:]  = icetgt[:,:,:]
NC_SIC_PDT[:,:,:]  = icepdt[:,:,:]
NC_SIC_ERR[1:,:,:] = iceerr[:,:,:]
NC_SIC_RMS[0,:,:]  = icerms[:,:]
NC_SIC_RMS_MEAN[0] = icerms_mean

nc_fout_id.close()

In [23]:
print("Mean RMSE =", icerms_mean)

Mean RMSE = 15.444548144766157
